# Análise de dados dos deputados

In [1]:
import pandas as pd

In [2]:
df_candidatos = pd.read_csv('../data/output/votos_consolidados_por_deputado.csv')

In [3]:
df_partidos = pd.read_csv('../data/output/votos_consolidados_por_partido.csv')

In [4]:
deputados_eleitos_camara = pd.read_csv('../data/output/deputados_eleitos.csv')

In [5]:
PARTIDO = 'PSL'
ESTADO = 'RJ'

In [6]:
df_candidatos.columns

Index(['composicao_legenda', 'descricao_detalhe_situacao_candidatura',
       'descricao_situacao_candidatura', 'descricao_totalizacao_turno',
       'descricao_ue', 'nome', 'nome_legenda', 'nome_partido', 'nome_urna',
       'numero_partido', 'numero_urna', 'sigla_partido', 'sigla_uf',
       'tipo_agremiacao', 'total_votos'],
      dtype='object')

In [7]:
df_partidos.columns

Index(['composicao_legenda', 'descricao_ue', 'nome_legenda', 'nome_partido',
       'numero_partido', 'sigla_partido', 'sigla_uf', 'tipo_agremiacao',
       'total_legenda', 'total_votos'],
      dtype='object')

In [8]:
df_candidatos['sigla_uf'].unique()

array(['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG',
       'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR',
       'RS', 'SC', 'SE', 'SP', 'TO'], dtype=object)

In [9]:
deputados_por_estado = df_candidatos[df_candidatos['sigla_uf'] == ESTADO]

In [10]:
partidos_por_estado = df_partidos[df_partidos['sigla_uf'] == ESTADO]

In [11]:
deputados_por_estado['descricao_totalizacao_turno'].unique()

array(['SUPLENTE', 'ELEITO POR QP', 'NAO ELEITO', 'ELEITO POR MEDIA'],
      dtype=object)

In [12]:
# quantidade de deputados eleitos no RJ
conditions = ['ELEITO POR QP', 'ELEITO POR MEDIA']
dp_eleitos_estado = len(deputados_por_estado[deputados_por_estado['descricao_totalizacao_turno'].isin(conditions)])

In [13]:
# 1) Qual o coeficiente de votos para um partido / coligação ter ao menos uma cadeira, dentro do RJ?
# 2) Quantos votos o partido PSL conseguiu? (Lembrando que no RJ, disputaram sozinhos - sem coligação)
# 3) Quantos votos o Helio Bolsonaro conseguiu?
# 4) Houve deputados puxados? Quais?

In [14]:
# 1) Qual o coeficiente de votos para um partido / coligação ter ao menos uma cadeira, dentro do RJ?
soma_votos_estado = sum(deputados_por_estado['total_votos']) + sum(partidos_por_estado['total_legenda'])
coeciente_eleitoral_estado = soma_votos_estado / dp_eleitos_estado

In [15]:
print('Summary')
print(f'Quantidade de votos válidos para legenda + candidatos {soma_votos_estado}')
print(f'Quantidade de candidatos eleitos {dp_eleitos_estado}')
print(f'Coeciente eleitoral para o {ESTADO} {coeciente_eleitoral_estado}')

Summary
Quantidade de votos válidos para legenda + candidatos 9615518
Quantidade de candidatos eleitos 46
Coeciente eleitoral para o RJ 209033.0


In [16]:
print(f'Assim, para um partido ter ao menos uma cadeira, era necessário ter ao menos, a quantidade de votos que queremos analisar, que no caso do RJ, é de {coeciente_eleitoral_estado} votos')

Assim, para um partido ter ao menos uma cadeira, era necessário ter ao menos, a quantidade de votos que queremos analisar, que no caso do RJ, é de 209033.0 votos


In [17]:
# 2) Quantos votos o partido PSL conseguiu?
dps_part = deputados_por_estado[deputados_por_estado['sigla_partido'] == PARTIDO]
partido = partidos_por_estado[partidos_por_estado['sigla_partido'] == PARTIDO]
votos_partido = sum(dps_part['total_votos']) + sum(partido['total_legenda'])
cadeiras = int(votos_partido / coeciente_eleitoral_estado)
print(f'PSL na eleição, para o RJ, conseguiu {votos_partido} votos, ou seja, {cadeiras} cadeiras')

PSL na eleição, para o RJ, conseguiu 1927215 votos, ou seja, 9 cadeiras


In [18]:
# 3) Quantos votos o Helio Bolsonaro conseguiu?
dps_eleitos = dps_part[dps_part['descricao_totalizacao_turno'].isin(conditions)].sort_values(by='total_votos', ascending=False)

In [19]:
mais_eleito = dps_eleitos.iloc[0]
print(f'O deputado mais eleito no {PARTIDO} foi {mais_eleito.nome_urna}, com {mais_eleito.total_votos} votos')

O deputado mais eleito no PSL foi HELIO FERNANDO BARBOSA LOPES, com 345234 votos


In [22]:
# 4) Houve deputados puxados? Quais?
# Primeiramente, como saber se houve deputados puxados? A primeira hipótese, é investigar casos em que os votos foram
# menores que o coeficiente eleitoral

props = ['nome_urna', 'total_votos', 'descricao_totalizacao_turno']
dps_eleitos[props]

,nome_urna,total_votos,descricao_totalizacao_turno
4419,HELIO FERNANDO BARBOSA LOPES,345234,ELEITO POR QP
4817,CARLOS JORDY,204048,ELEITO POR QP
4422,LUIZ LIMA,115119,ELEITO POR QP
4172,DELEGADO ANTONIO FURTADO,104211,ELEITO POR QP
4258,SARGENTO GURGEL,62089,ELEITO POR QP
4325,MAJOR FABIANA,57611,ELEITO POR QP
4690,FELICIO LATERCA,47065,ELEITO POR QP
4829,MARCIO LABRE,46934,ELEITO POR QP
4311,LOURIVAL GOMES,41307,ELEITO POR QP
4804,CHRIS TONIETTO,38525,ELEITO POR QP


In [23]:
dps_eleitos[dps_eleitos['total_votos'] <= coeciente_eleitoral_estado][props]

,nome_urna,total_votos,descricao_totalizacao_turno
4817,CARLOS JORDY,204048,ELEITO POR QP
4422,LUIZ LIMA,115119,ELEITO POR QP
4172,DELEGADO ANTONIO FURTADO,104211,ELEITO POR QP
4258,SARGENTO GURGEL,62089,ELEITO POR QP
4325,MAJOR FABIANA,57611,ELEITO POR QP
4690,FELICIO LATERCA,47065,ELEITO POR QP
4829,MARCIO LABRE,46934,ELEITO POR QP
4311,LOURIVAL GOMES,41307,ELEITO POR QP
4804,CHRIS TONIETTO,38525,ELEITO POR QP
4250,PROFESSOR JOZIEL,34274,ELEITO POR MEDIA
